In [ ]:
import sys
sys.path.append('../')

In [1]:
import csv
import pickle
import time

import numpy as np
import pandas as pd

from func import city_state_coord

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.options.mode.copy_on_write = True
from Cython.Build import cythonize
from geopy.distance import geodesic
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm as tqdmn

%load_ext Cython

In [2]:
url = 'https://www.bu.edu/brand/guidelines/editorial-style/us-state-abbreviations/'
tables = pd.read_html(url)

short_state_name =dict(zip(tables[0][0][1:],tables[0][2][1:]))
print(short_state_name)


url = 'https://www.americancities.ru/info/capital_cities/'
tables = pd.read_html(url)
capital_state = tables[0][['Почтовое сокращение','Столица штата']]
capital_state.columns = ['state','capital']
capital_state.head()

rows_to_append = [{'state':  'DC', 'capital': 'Washington'}]

capital_state = pd.concat([capital_state,pd.DataFrame(rows_to_append)],axis=0,ignore_index=True)
capital_state

{'Alabama': 'AL', 'Alaska': 'AK', 'Arkansas': 'AR', 'Arizona': 'AZ', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


,state,capital
0,AL,Montgomery
1,AK,Juneau
2,AZ,Phoenix
3,AR,Little Rock
4,CA,Sacramento
5,CO,Denver
6,CT,Hartford
7,DE,Dover
8,FL,Tallahassee
9,GA,Atlanta


In [3]:
capital_state.columns = ['state', 'city']

In [4]:
capital_state = city_state_coord(capital_state)

  0%|          | 0/51 [00:00<?, ?it/s]

In [5]:
capital_state

,state,city,lat,long
0,AL,Montgomery,32.34379895,-86.2460375
1,AK,Juneau,58.38502345,-134.197786
2,AZ,Phoenix,33.6054149,-112.125051
3,AR,Little Rock,34.723749,-92.336122
4,CA,Sacramento,38.56173395,-121.46143
5,CO,Denver,39.76433895,-104.85511145
6,CT,Hartford,41.76568745,-72.68008695
7,DE,Dover,39.1564159,-75.51341995
8,FL,Tallahassee,30.4671395,-84.2568559
9,GA,Atlanta,33.7673675,-84.42013055


In [6]:
capital_state.columns = ['state', 'capital','lat_capital', 'long_capital']

In [7]:
# Самые крупные города США
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
tables = pd.read_html(url)
tables = tables[2]
tables.columns
print(tables)

            City  ST 2023 estimate 2020 census  Change 2020 land area          \
            City  ST 2023 estimate 2020 census  Change            mi2     km2   
0    New York[c]  NY       8258035     8804190  −6.20%          300.5   778.3   
1    Los Angeles  CA       3820914     3898747  −2.00%          469.5  1216.0   
2        Chicago  IL       2664452     2746388  −2.98%          227.7   589.7   
3        Houston  TX       2314157     2304580  +0.42%          640.4  1658.6   
4        Phoenix  AZ       1650070     1608139  +2.61%          518.0  1341.6   
..           ...  ..           ...         ...     ...            ...     ...   
331         Yuma  AZ        100858       95548  +5.56%          120.7   312.6   
332  New Bedford  MA        100695      101079  −0.38%           20.0    51.8   
333   Suffolk[l]  VA        100659       94324  +6.72%          399.2  1033.9   
334     Hesperia  CA        100633       99818  +0.82%           72.7   188.3   
335    Davenport  IA        

In [8]:
major_city = tables[['City','ST','2023 estimate', ]].copy()
major_city.columns = ['major_city','state','2023_estimate',]
major_city['major_city'] = major_city['major_city'].apply(lambda x: x.split(sep='[')[0])
major_city.columns = ['city', 'state', '2023_estimate']
major_city.head()

,city,state,2023_estimate
0,New York,NY,8258035
1,Los Angeles,CA,3820914
2,Chicago,IL,2664452
3,Houston,TX,2314157
4,Phoenix,AZ,1650070


In [9]:
million_city = major_city[major_city['2023_estimate']>=10**6][['city','state']]
million_city = city_state_coord(million_city)
million_city.columns = ['million_city','state','lat_million_city','long_million_city']
million_city

  0%|          | 0/9 [00:00<?, ?it/s]

,million_city,state,lat_million_city,long_million_city
0,New York,NY,40.6974881,-73.979681
1,Los Angeles,CA,34.020479,-118.4117325
2,Chicago,IL,41.83373295,-87.73196395
3,Houston,TX,29.81717795,-95.40129145
4,Phoenix,AZ,33.6054149,-112.125051
5,Philadelphia,PA,40.002498,-75.1180283
6,San Antonio,TX,29.4585735,-98.51437645
7,San Diego,CA,32.8245525,-117.10897845
8,Dallas,TX,32.82066455,-96.73155255


In [10]:
with open('data/mln_city.pkl', 'rb') as pkl_file:
        mln_city = pickle.load(pkl_file)
mln_city.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   city               9 non-null      object
 1   state              9 non-null      object
 2   lat_million_city   9 non-null      object
 3   long_million_city  9 non-null      object
dtypes: object(4)
memory usage: 360.0+ bytes


In [11]:
mask1 = major_city['2023_estimate']<=10**6
mask2 = major_city['2023_estimate']>=3*10**5
big_city = major_city[mask1&mask2][['city','state']]
big_city = city_state_coord(big_city)
big_city.columns = ['big_city','state','lat_big_city','long_big_city']
big_city

  0%|          | 0/60 [00:00<?, ?it/s]

,big_city,state,lat_big_city,long_big_city
9,Jacksonville,FL,30.34499,-81.6831069
10,Austin,TX,30.3077609,-97.75574245
11,Fort Worth,TX,32.80081295,-97.28931895
12,San Jose,CA,37.2959622,-121.8160962
13,Columbus,OH,39.98295145,-82.990829
14,Charlotte,NC,35.20315345,-80.83982895
15,Indianapolis,IN,39.7797845,-86.13275
16,San Francisco,CA,37.7577,-122.4376
17,Seattle,WA,47.608715,-122.33979795
18,Denver,CO,39.76433895,-104.85511145


In [12]:
mask1 = major_city['2023_estimate']>=10**5
mask2 = major_city['2023_estimate']<=3*10**5
large_city = major_city[mask1&mask2][['city','state']]
large_city = city_state_coord(large_city)
large_city.columns = ['large_city','state','lat_large_ctiy','long_large_ctiy']
large_city

  0%|          | 0/267 [00:00<?, ?it/s]

,large_city,state,lat_large_ctiy,long_large_ctiy
69,Durham,NC,36.0018269,-78.884911
70,Lincoln,NE,40.80436265,-96.68580995
71,Jersey City,NJ,40.7152445,-74.06873095
72,Plano,TX,33.061262,-96.7366254
73,Anchorage,AK,61.10886445,-149.4403109
...,...,...,...,...
331,Yuma,AZ,32.60568045,-114.572058
332,New Bedford,MA,41.6637935,-70.92791595
333,Suffolk,VA,36.7461174,-76.6652514
334,Hesperia,CA,34.38784095,-117.33865995


In [13]:
# сохранение
# Производим сериализацию и записываем результат в файл формата pkl
with open('data/capital_state.pkl', 'wb') as output:
    pickle.dump(capital_state, output)


capital_state.to_csv('data/capital_state.csv',index_label=False)

with open('data/lg_city.pkl', 'wb') as output:
    pickle.dump(large_city, output)


large_city.to_csv('data/lg_city.csv', index_label=False)

with open('data/bg_city.pkl', 'wb') as output:
    pickle.dump(big_city, output)


big_city.to_csv('data/bg_city.csv', index_label=False)

with open('data/mln_city.pkl', 'wb') as output:
    pickle.dump(million_city, output)


million_city.to_csv('data/mln_city.csv', index_label=False)